In [2]:
%%bash
pip install pandas-datareader
pip install get-all-tickers

  Created wheel for get-all-tickers: filename=get_all_tickers-1.7-cp36-none-any.whl size=4242 sha256=9af489b20dccd752bfaa425f8e488e92109e7e4924356ae3b776a9df769a7b80
  Stored in directory: /root/.cache/pip/wheels/d0/8b/ef/6cd3580f4b479aef881a32bd937c282982e3d186b617a663ac
Successfully built get-all-tickers


In [3]:

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History
from collections import Iterable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import pandas as pd
import time
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
from get_all_tickers import get_tickers as gt


In [4]:
#getting the labels
news_data = pd.read_csv('Combined_News_DJIA.csv', index_col=0)
labels = news_data[['Label']]
labels.head(5)

,Label
Date,
2008-08-08,0
2008-08-11,1
2008-08-12,0
2008-08-13,0
2008-08-14,1


# Ticker - NKTR

In [5]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL
list_of_tickers=['NKTR'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

,High,Low,Open,Volume,Adj Close
Date,,,,,
2008-08-08,4.79,4.43,4.43,1031200.0,4.79
2008-08-11,4.93,4.64,4.81,704400.0,4.83
2008-08-12,4.83,4.60,4.81,935600.0,4.67
2008-08-13,4.77,4.48,4.66,1101000.0,4.72
2008-08-14,5.03,4.68,4.69,1123400.0,5.01
...,...,...,...,...,...
2016-06-27,14.33,13.31,14.23,1160200.0,13.39
2016-06-28,13.93,13.54,13.58,1069700.0,13.76
2016-06-29,14.30,13.59,13.99,752000.0,13.97


In [6]:
with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)
with_sentiment_data

,Label,High,Low,Open,Volume,Adj Close
Date,,,,,,
2008-08-08,0,4.79,4.43,4.43,1031200.0,4.79
2008-08-11,1,4.93,4.64,4.81,704400.0,4.83
2008-08-12,0,4.83,4.60,4.81,935600.0,4.67
2008-08-13,0,4.77,4.48,4.66,1101000.0,4.72
2008-08-14,1,5.03,4.68,4.69,1123400.0,5.01
...,...,...,...,...,...,...
2016-06-27,0,14.33,13.31,14.23,1160200.0,13.39
2016-06-28,1,13.93,13.54,13.58,1069700.0,13.76
2016-06-29,1,14.30,13.59,13.99,752000.0,13.97


# Standardization

In [7]:
#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])



In [18]:
scaled_wo_sentiment

,Open,High,Low,Volume,Adj Close
0,-1.658294,-1.676088,-1.719641,-0.074268,-1.613051
1,-1.617279,-1.612689,-1.606773,-0.443055,-1.601171
2,-1.646575,-1.624765,-1.606773,-0.182150,-1.648691
3,-1.664153,-1.660993,-1.651326,0.004500,-1.633841
4,-1.587983,-1.600613,-1.642415,0.029778,-1.547710
...,...,...,...,...,...
1984,1.136545,1.004774,1.191169,0.071306,0.941174
1985,1.019361,1.074210,0.998106,-0.030821,1.051065
1986,1.127756,1.089305,1.119884,-0.389339,1.113435
1987,1.107249,1.131571,1.108004,-0.116360,1.190656


# Train and Test split 

In [19]:
x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)

In [10]:
class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [11]:
def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)

# Creating time lags for time series prediction

In [20]:
TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)

In [21]:

x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))

# LSTM model for Evaluation without sentiment

In [22]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

CPU times: user 38.5 s, sys: 2.45 s, total: 40.9 s
Wall time: 27.6 s


In [23]:
time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.239185571670532
Base MSE for the above model: 0.004376023076474667


# LSTM model for Evaluation with sentiment

In [24]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

CPU times: user 39.2 s, sys: 2.65 s, total: 41.9 s
Wall time: 28.4 s


In [25]:
time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.05492877960205
Base MSE for the above model: 0.0036584073677659035


# Comparison of MSE with Sentiment and without Sentiment scores

In [27]:
d = {'MSE Without using Sentiment Score': [0.0043], 'MSE With Sentiment Score': [0.0036]}
df_comparison1 = pd.DataFrame(data=d)
df_comparison1

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0043,0.0036


# Ticker - AMD

In [28]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL
list_of_tickers=['AMD'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

,High,Low,Open,Volume,Adj Close
Date,,,,,
2008-08-08,5.26,5.05,5.19,22304100,5.13
2008-08-11,5.20,5.03,5.10,15651300,5.11
2008-08-12,5.35,5.12,5.20,30836400,5.21
2008-08-13,5.23,5.00,5.20,19108700,5.15
2008-08-14,5.37,5.14,5.15,18960500,5.30
...,...,...,...,...,...
2016-06-27,5.05,4.65,4.88,31025300,4.72
2016-06-28,5.19,4.91,4.95,29221400,5.12
2016-06-29,5.40,5.10,5.31,33118300,5.13


In [30]:
with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))

# LSTM Model for evaluation without the sentiment

In [34]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.534465551376343
Base MSE for the above model: 0.002150504384189844
CPU times: user 39.9 s, sys: 2.32 s, total: 42.2 s
Wall time: 27.9 s


# LSTM Model for evaluation with the sentiment

In [35]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.21262001991272
Base MSE for the above model: 0.0017484786221757531
CPU times: user 39.4 s, sys: 2.3 s, total: 41.7 s
Wall time: 27.6 s


# Comparison of MSE with Sentiment and without Sentiment scores

In [36]:
d = {'MSE Without using Sentiment Score': [0.0021], 'MSE With Sentiment Score': [0.0017]}
df_comparison2 = pd.DataFrame(data=d)
df_comparison2

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0021,0.0017


# Ticker - KGC

In [37]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL
list_of_tickers=['KGC'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM for Evaluation without sentiment

In [38]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.80247950553894
Base MSE for the above model: 0.004240995738655329
CPU times: user 40.1 s, sys: 2.35 s, total: 42.4 s
Wall time: 28.1 s


# LSTM for Evaluation with sentiment

In [39]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model = Sequential()
lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model.add(keras.layers.Dense(units = 1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.028542280197144
Base MSE for the above model: 0.00516292080283165
CPU times: user 39 s, sys: 2.17 s, total: 41.2 s
Wall time: 27.4 s


In [40]:
d = {'MSE Without using Sentiment Score': [0.0051], 'MSE With Sentiment Score': [0.0042]}
df_comparison3 = pd.DataFrame(data=d)
df_comparison3

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0051,0.0042


# Ticker - Zion

In [41]:
##ZION,SOHU,BLIN,WMT,AMGN,T,AAPL
list_of_tickers=['ZION'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM for prediction without the sentiment score

In [52]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_zion = Sequential()
lstm_model_zion.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_zion.add(keras.layers.Dense(units = 1))
lstm_model_zion.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_zion.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.982330560684204
Base MSE for the above model: 0.0012966925278306007
CPU times: user 40.2 s, sys: 2.39 s, total: 42.6 s
Wall time: 28.3 s


# LSTM using the sentiment scores

In [53]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_zion1 = Sequential()
lstm_model_zion1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_zion1.add(keras.layers.Dense(units = 1))
lstm_model_zion1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_zion1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.19249415397644
Base MSE for the above model: 0.0013115211622789502
CPU times: user 39.3 s, sys: 2.24 s, total: 41.6 s
Wall time: 27.5 s


In [54]:
d = {'MSE Without using Sentiment Score': [0.0012], 'MSE With Sentiment Score': [0.0013]}
df_comparison4 = pd.DataFrame(data=d)
df_comparison4

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0012,0.0013


# Ticker - AMGN

In [45]:
##SOHU,BLIN,WMT,AMGN,T,AAPL
list_of_tickers=['AMGN'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM evaluation without sentiment

In [50]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_amgn = Sequential()
lstm_model_amgn.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_amgn.add(keras.layers.Dense(units = 1))
lstm_model_amgn.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_amgn.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.57383966445923
Base MSE for the above model: 0.0016074971063062549
CPU times: user 39.9 s, sys: 2.19 s, total: 42.1 s
Wall time: 27.9 s


# LSTM evaluation with sentiment

In [49]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model1 = Sequential()
lstm_model1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model1.add(keras.layers.Dense(units = 1))
lstm_model1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.62963342666626
Base MSE for the above model: 0.001529995002783835
CPU times: user 40 s, sys: 2.35 s, total: 42.4 s
Wall time: 28 s


In [51]:
d = {'MSE Without using Sentiment Score': [0.0016], 'MSE With Sentiment Score': [0.0015]}
df_comparison5 = pd.DataFrame(data=d)
df_comparison5

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0016,0.0015


# Ticker - SOHU

In [55]:
##,BLIN,WMT,T,AAPL
list_of_tickers=['SOHU'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM evaluation without using the sentiment scores 

In [57]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_sohu = Sequential()
lstm_model_sohu.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_sohu.add(keras.layers.Dense(units = 1))
lstm_model_sohu.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_sohu.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.07793951034546
Base MSE for the above model: 0.006253760773688555
CPU times: user 40.5 s, sys: 2.48 s, total: 42.9 s
Wall time: 28.5 s


# LSTM evaluation using the sentiment

In [58]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_sohu1 = Sequential()
lstm_model_sohu1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_sohu1.add(keras.layers.Dense(units = 1))
lstm_model_sohu1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_sohu1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.23858404159546
Base MSE for the above model: 0.006465512793511152
CPU times: user 40.5 s, sys: 2.38 s, total: 42.9 s
Wall time: 28.6 s


In [59]:
d = {'MSE Without using Sentiment Score': [0.0062], 'MSE With Sentiment Score': [0.0064]}
df_comparison6 = pd.DataFrame(data=d)
df_comparison6

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0062,0.0064


# Ticker - T

In [60]:
##,BLIN,WMT,T,AAPL
list_of_tickers=['T'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM Evaluation without sentiment

In [61]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_t = Sequential()
lstm_model_t.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_t.add(keras.layers.Dense(units = 1))
lstm_model_t.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_t.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.035260915756226
Base MSE for the above model: 0.0915767177939415
CPU times: user 40.8 s, sys: 2.2 s, total: 43 s
Wall time: 28.5 s


# LSTM evaluation with sentiment

In [62]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_t1 = Sequential()
lstm_model_t1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_t1.add(keras.layers.Dense(units = 1))
lstm_model_t1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_t1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.730696439743042
Base MSE for the above model: 0.09170409291982651
CPU times: user 40.1 s, sys: 2.31 s, total: 42.4 s
Wall time: 28.1 s


In [63]:
d = {'MSE Without using Sentiment Score': [0.0091], 'MSE With Sentiment Score': [0.0091]}
df_comparison7 = pd.DataFrame(data=d)
df_comparison7

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0091,0.0091


# Ticker -AAPL

In [64]:
##,BLIN,WMT,AAPL
list_of_tickers=['AAPL'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM evaluation without sentiment scores

In [65]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_ap = Sequential()
lstm_model_ap.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_ap.add(keras.layers.Dense(units = 1))
lstm_model_ap.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_ap.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.980021715164185
Base MSE for the above model: 0.003923584241420031
CPU times: user 40.4 s, sys: 2.33 s, total: 42.8 s
Wall time: 28.3 s


# LSTM evaluation with sentiment scores

In [66]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_ap1 = Sequential()
lstm_model_ap1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_ap1.add(keras.layers.Dense(units = 1))
lstm_model_ap1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_ap1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.52291989326477
Base MSE for the above model: 0.003186717163771391
CPU times: user 40.7 s, sys: 2.51 s, total: 43.2 s
Wall time: 28.9 s


In [67]:
d = {'MSE Without using Sentiment Score': [0.0039], 'MSE With Sentiment Score': [0.0031]}
df_comparison8 = pd.DataFrame(data=d)
df_comparison8

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0039,0.0031


# Ticker - BLIN

In [68]:
##BLIN,WMT
list_of_tickers=['BLIN'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM evaluation without sentiment

In [69]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_bl = Sequential()
lstm_model_bl.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_bl.add(keras.layers.Dense(units = 1))
lstm_model_bl.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_bl.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.43800139427185
Base MSE for the above model: 0.007138579618185759
CPU times: user 39.6 s, sys: 2.3 s, total: 41.9 s
Wall time: 27.8 s


# LSTM evaluation with sentiment

In [70]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_bl1 = Sequential()
lstm_model_bl1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_bl1.add(keras.layers.Dense(units = 1))
lstm_model_bl1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_bl1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  27.422298908233643
Base MSE for the above model: 0.007361146155744791
CPU times: user 39.7 s, sys: 2.23 s, total: 42 s
Wall time: 27.8 s


In [71]:
d = {'MSE Without using Sentiment Score': [0.0071], 'MSE With Sentiment Score': [0.0073]}
df_comparison9 = pd.DataFrame(data=d)
df_comparison9

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0071,0.0073


# Ticker - WMT

In [75]:
##WMT
list_of_tickers=['WMT'] 

for i in list_of_tickers:
  start = datetime.datetime(2008, 8, 8)
  end = datetime.datetime(2016, 7, 1)
  zion = web.DataReader(i, 'yahoo', start, end)
data=zion.drop(['Close'],axis=1)
temp_data=zion.drop(['Close'],axis=1)
temp_data

with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)

#Scaling the variables(x) and variable(y)
sca = temp_data.iloc[:,:]

#Standardization of data is required as the dataset consists of variables of different scales
sc = StandardScaler()
scaled_wo_sentiment = sc.fit_transform(sca)
scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])

with_sentiment_data = sc.fit_transform(with_sentiment_data)
with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])

x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]

x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
#Splitting data into training(70% data) and testing data(30% data)
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)

x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)


class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)


TIME_STEPS = 1

#creating lags for with sentiment scores

x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)

#creating lags for without sentiment scores

x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)



x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))

x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))



# LSTM Evaluation without sentiment

In [76]:
#Evaluation using without sentiment
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_wm = Sequential()
lstm_model_wm.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
lstm_model_wm.add(keras.layers.Dense(units = 1))
lstm_model_wm.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_wm.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)


time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  28.465714693069458
Base MSE for the above model: 0.029594529420137405
CPU times: user 40.5 s, sys: 2.34 s, total: 42.8 s
Wall time: 28.8 s


# LSTM Evaluation with sentiment

In [77]:
#now evaluating the performance of the model using the sentiment labels

#Evaluation using the Expertise
%%time
np.random.seed = 1   
tf.random.set_seed = 2  
time3=fun_t()
history=History()
lstm_model_wm1 = Sequential()
lstm_model_wm1.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
lstm_model_wm1.add(keras.layers.Dense(units = 1))
lstm_model_wm1.compile(loss='mean_squared_error', optimizer='adam')
history = lstm_model_wm1.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)

time_lstm = list(time3.times)
print('Long Short Term Memory Model Time Taken: ',sum(time_lstm))
train_lstm = history.history.get('loss')
val_lstm = history.history.get('val_loss')

base_mse=val_lstm[-1]
print('Base MSE for the above model:',base_mse)

Long Short Term Memory Model Time Taken:  26.94499135017395
Base MSE for the above model: 0.03206600248813629
CPU times: user 38.9 s, sys: 2.24 s, total: 41.2 s
Wall time: 27.3 s


In [78]:
d = {'MSE Without using Sentiment Score': [0.0295], 'MSE With Sentiment Score': [0.032]}
df_comparison10 = pd.DataFrame(data=d)
df_comparison10

,MSE Without using Sentiment Score,MSE With Sentiment Score
0,0.0295,0.032
